In [1]:
from pyspark.sql import SparkSession

In [2]:
# Étape 2 : Créer la session Spark avec les drivers JDBC
spark = SparkSession.builder \
    .appName("Test MySQL Ingestion") \
    .getOrCreate()


In [3]:

#  Étape 3 : Lire la table MySQL
df_mysql = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://mysql:3306/hospital_legacy") \
    .option("dbtable", "patients_legacy") \
    .option("user", "source_reader") \
    .option("password", "Source_Read_2024!") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()

In [4]:
df_mysql.show(50)

+----------+------+---------+--------------+----+--------------------+-----------+-----------+---------+-------------+
|patient_id|   nom|   prenom|date_naissance|sexe|             adresse|      ville|code_postal|telephone|date_creation|
+----------+------+---------+--------------+----+--------------------+-----------+-----------+---------+-------------+
|         1|Diallo|  Aminata|    1985-06-12|   F|Rue des Manguiers 12|      Dakar|      11000|771234567|   2020-01-15|
|         2|    Ba|   Moussa|    1978-03-22|   M|Avenue Blaise Diagne|     ThiÃ¨s|      21000|772345678|   2019-11-03|
|         3|   Sow|    Fatou|    1990-09-10|   F|      Rue 4, MÃ©dina|      Dakar|      11000|773456789|   2021-05-20|
|         4|Ndiaye| Ibrahima|    1982-12-01|   M|   CitÃ© Keur Gorgui|      Dakar|      11000|774567890|   2018-07-12|
|         5|  Fall|      Awa|    1995-04-18|   F|     Rue des Jardins|Saint-Louis|      32000|775678901|   2022-02-28|
|         6|  Diop|   Cheikh|    1975-08-30|   M

In [5]:
# Lire le fichier CSV
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/jovyan/work/data/patients_source.csv")

# Afficher les premières lignes
df_csv.show(10)

+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+
|external_id|   nom|  prenom|date_naissance|sexe|               email|     num_secu|source_system|date_import|
+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+
|     SRC001|Diallo| Aminata|    12/06/1985|   F|aminata.diallo@ex...|2850612345678|   SOURCE_CSV| 2024-01-01|
|     SRC002|    Ba|  Moussa|    22/03/1978|   M|moussa.ba@example...|2780322456789|   SOURCE_CSV| 2024-01-01|
|     SRC003|   Sow|   Fatou|    10/09/1990|   F|fatou.sow@example...|2900912345678|   SOURCE_CSV| 2024-01-01|
|     SRC004|Ndiaye|Ibrahima|    01/12/1982|   M|ibrahima.ndiaye@e...|2821212345678|   SOURCE_CSV| 2024-01-01|
|     SRC005|  Fall|     Awa|    18/04/1995|   F|awa.fall@example.com|2950412345678|   SOURCE_CSV| 2024-01-01|
|     SRC006|  Diop|  Cheikh|    30/08/1975|   M|cheikh.diop@examp...|2750812345678|   SOURCE_CSV| 2024-01-01|
|

In [40]:
import requests
import pandas as pd

url = "http://api-medical:8000/visits"

response = requests.get(url)

if response.status_code == 200:
    
    data = response.json()
    
    # EXTRAIRE la liste visits
    df_api = pd.DataFrame(data["visits"])
    
    display(df_api)

else:
    print(f"Erreur {response.status_code} : {response.text}")


,visit_id,patient_id,date_visite,type_visite,motif,diagnostic,medecin,duree_minutes
0,8,8,2024-01-12T09:00:00,consultation,Bilan sanguin,Anémie légère,Dr. Gaye,20
1,7,7,2024-01-11T13:00:00,consultation,Vaccination,RAS,Dr. Camara,10
2,6,6,2024-01-10T08:30:00,urgence,Douleurs thoraciques,Angine,Dr. Ndiaye,35
3,5,5,2024-01-09T16:45:00,consultation,Fièvre,Grippe,Dr. Faye,15
4,4,4,2024-01-08T10:00:00,consultation,Suivi grossesse,Grossesse normale,Dr. Diouf,30
5,3,3,2024-01-07T14:15:00,urgence,Douleurs abdominales,Gastro-entérite,Dr. Sow,40
6,2,2,2024-01-06T11:00:00,consultation,Toux persistante,Bronchite,Dr. Ba,25
7,1,1,2024-01-05T09:30:00,consultation,Fièvre,Paludisme,Dr. Ndiaye,20


#### Configuration pour la connexion de MINIO

In [7]:
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", "http://medical-minio:9000")
hadoop_conf.set("fs.s3a.access.key", "medicaladmin")
hadoop_conf.set("fs.s3a.secret.key", "MedicalSecurePass123!")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [8]:
bronze_path = "s3a://bronze"

In [10]:
# 1. MySQL → MinIO 
df_mysql.write.mode("overwrite").parquet(f"{bronze_path}/patients_legacy")

In [11]:
# 2. CSV → MinIO 
df_csv.write.mode("overwrite").parquet(f"{bronze_path}/patients_source")

In [41]:

spark = SparkSession.builder.getOrCreate()

# Conversion Pandas → Spark
df_api_spark = spark.createDataFrame(df_api)

# Écriture dans MinIO
df_api_spark.coalesce(1).write.mode("overwrite").parquet(f"{bronze_path}/visits")

## 🧱 Étapes de transformation Silver

##### Lire les données Bronze depuis MinIO

In [29]:
### Lire depuis le fichier patients_source

bronze_path = "s3a://bronze"
df_patient_source = spark.read.parquet(f"{bronze_path}/patients_source")
df_patient_source.show()

+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+
|external_id|   nom|  prenom|date_naissance|sexe|               email|     num_secu|source_system|date_import|
+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+
|     SRC001|Diallo| Aminata|    12/06/1985|   F|aminata.diallo@ex...|2850612345678|   SOURCE_CSV| 2024-01-01|
|     SRC002|    Ba|  Moussa|    22/03/1978|   M|moussa.ba@example...|2780322456789|   SOURCE_CSV| 2024-01-01|
|     SRC003|   Sow|   Fatou|    10/09/1990|   F|fatou.sow@example...|2900912345678|   SOURCE_CSV| 2024-01-01|
|     SRC004|Ndiaye|Ibrahima|    01/12/1982|   M|ibrahima.ndiaye@e...|2821212345678|   SOURCE_CSV| 2024-01-01|
|     SRC005|  Fall|     Awa|    18/04/1995|   F|awa.fall@example.com|2950412345678|   SOURCE_CSV| 2024-01-01|
|     SRC006|  Diop|  Cheikh|    30/08/1975|   M|cheikh.diop@examp...|2750812345678|   SOURCE_CSV| 2024-01-01|
|

In [30]:
### Lire depuis le fichier patient legacy

df_patient_legacy = spark.read.parquet(f"{bronze_path}/patients_legacy")
df_patient_legacy.show()

+----------+------+---------+--------------+----+--------------------+-----------+-----------+---------+-------------+
|patient_id|   nom|   prenom|date_naissance|sexe|             adresse|      ville|code_postal|telephone|date_creation|
+----------+------+---------+--------------+----+--------------------+-----------+-----------+---------+-------------+
|         1|Diallo|  Aminata|    1985-06-12|   F|Rue des Manguiers 12|      Dakar|      11000|771234567|   2020-01-15|
|         2|    Ba|   Moussa|    1978-03-22|   M|Avenue Blaise Diagne|     ThiÃ¨s|      21000|772345678|   2019-11-03|
|         3|   Sow|    Fatou|    1990-09-10|   F|      Rue 4, MÃ©dina|      Dakar|      11000|773456789|   2021-05-20|
|         4|Ndiaye| Ibrahima|    1982-12-01|   M|   CitÃ© Keur Gorgui|      Dakar|      11000|774567890|   2018-07-12|
|         5|  Fall|      Awa|    1995-04-18|   F|     Rue des Jardins|Saint-Louis|      32000|775678901|   2022-02-28|
|         6|  Diop|   Cheikh|    1975-08-30|   M

#### Nettoyage de df_patient_source

Convertir date_naissance au bon format

Supprimer les lignes incomplètes

Supprimer les doublons

In [31]:
from pyspark.sql.functions import col, to_date, upper, trim

df_source_clean = df_patient_source \
    .filter(col("external_id").isNotNull() & col("nom").isNotNull() & col("prenom").isNotNull()) \
    .withColumn("date_naissance", to_date(col("date_naissance"), "dd/MM/yyyy")) \
    .withColumn("nom", upper(trim(col("nom")))) \
    .withColumn("prenom", upper(trim(col("prenom")))) \
    .dropDuplicates(["external_id"])

In [32]:
df_patient_source.show()

+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+
|external_id|   nom|  prenom|date_naissance|sexe|               email|     num_secu|source_system|date_import|
+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+
|     SRC001|Diallo| Aminata|    12/06/1985|   F|aminata.diallo@ex...|2850612345678|   SOURCE_CSV| 2024-01-01|
|     SRC002|    Ba|  Moussa|    22/03/1978|   M|moussa.ba@example...|2780322456789|   SOURCE_CSV| 2024-01-01|
|     SRC003|   Sow|   Fatou|    10/09/1990|   F|fatou.sow@example...|2900912345678|   SOURCE_CSV| 2024-01-01|
|     SRC004|Ndiaye|Ibrahima|    01/12/1982|   M|ibrahima.ndiaye@e...|2821212345678|   SOURCE_CSV| 2024-01-01|
|     SRC005|  Fall|     Awa|    18/04/1995|   F|awa.fall@example.com|2950412345678|   SOURCE_CSV| 2024-01-01|
|     SRC006|  Diop|  Cheikh|    30/08/1975|   M|cheikh.diop@examp...|2750812345678|   SOURCE_CSV| 2024-01-01|
|

#### Nettoyage de patients_legacy

In [33]:
df_legacy_clean = df_patient_legacy \
    .withColumn("nom", upper(trim(col("nom")))) \
    .withColumn("prenom", upper(trim(col("prenom")))) \
    .withColumn("date_naissance", to_date(col("date_naissance"), "dd/MM/yyyy"))


#### Jointure

#### jointure sur nom, prenom, date_naissance pour enrichir les données source avec les infos manquantes de la legacy

In [34]:
df_silver = df_source_clean.join(
    df_legacy_clean,
    on=[
        df_source_clean.nom == df_legacy_clean.nom,
        df_source_clean.prenom == df_legacy_clean.prenom,
        df_source_clean.date_naissance == df_legacy_clean.date_naissance
    ],
    how="left"
).drop(df_legacy_clean.nom).drop(df_legacy_clean.prenom).drop(df_legacy_clean.date_naissance).drop(df_legacy_clean.sexe)

In [35]:
df_silver.show()

+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+----------+--------------------+-----------+-----------+---------+-------------+
|external_id|   nom|  prenom|date_naissance|sexe|               email|     num_secu|source_system|date_import|patient_id|             adresse|      ville|code_postal|telephone|date_creation|
+-----------+------+--------+--------------+----+--------------------+-------------+-------------+-----------+----------+--------------------+-----------+-----------+---------+-------------+
|     SRC001|DIALLO| AMINATA|    1985-06-12|   F|aminata.diallo@ex...|2850612345678|   SOURCE_CSV| 2024-01-01|         1|Rue des Manguiers 12|      Dakar|      11000|771234567|   2020-01-15|
|     SRC002|    BA|  MOUSSA|    1978-03-22|   M|moussa.ba@example...|2780322456789|   SOURCE_CSV| 2024-01-01|         2|Avenue Blaise Diagne|     ThiÃ¨s|      21000|772345678|   2019-11-03|
|     SRC003|   SOW|   FATOU|    1990-09-10| 

#### Sélection des colonnes finales

In [37]:
df_patients_silver = df_silver.select(
    "external_id",
    "patient_id",
    "nom",
    "prenom",
    "date_naissance",
    "email",
    "num_secu",
    "adresse",
    "sexe",
    "ville",
    "code_postal",
    "telephone",
    "date_creation"
)


In [38]:
df_patients_silver.show()

+-----------+----------+------+--------+--------------+--------------------+-------------+--------------------+----+-----------+-----------+---------+-------------+
|external_id|patient_id|   nom|  prenom|date_naissance|               email|     num_secu|             adresse|sexe|      ville|code_postal|telephone|date_creation|
+-----------+----------+------+--------+--------------+--------------------+-------------+--------------------+----+-----------+-----------+---------+-------------+
|     SRC001|         1|DIALLO| AMINATA|    1985-06-12|aminata.diallo@ex...|2850612345678|Rue des Manguiers 12|   F|      Dakar|      11000|771234567|   2020-01-15|
|     SRC002|         2|    BA|  MOUSSA|    1978-03-22|moussa.ba@example...|2780322456789|Avenue Blaise Diagne|   M|     ThiÃ¨s|      21000|772345678|   2019-11-03|
|     SRC003|         3|   SOW|   FATOU|    1990-09-10|fatou.sow@example...|2900912345678|      Rue 4, MÃ©dina|   F|      Dakar|      11000|773456789|   2021-05-20|
|     SRC0

#### Stocker les resultats dans MinIO (Silver)

In [39]:
silver_path = "s3a://silver/patients"

df_patients_silver.write.mode("overwrite").parquet(silver_path)

In [46]:
### Lire depuis le fichier patient legacy

df_visits = spark.read.parquet(f"{bronze_path}/visits")
df_visits.show()

+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+
|visit_id|patient_id|        date_visite| type_visite|               motif|       diagnostic|   medecin|duree_minutes|
+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+
|       8|         8|2024-01-12T09:00:00|consultation|       Bilan sanguin|    Anémie légère|  Dr. Gaye|           20|
|       7|         7|2024-01-11T13:00:00|consultation|         Vaccination|              RAS|Dr. Camara|           10|
|       6|         6|2024-01-10T08:30:00|     urgence|Douleurs thoraciques|           Angine|Dr. Ndiaye|           35|
|       5|         5|2024-01-09T16:45:00|consultation|              Fièvre|           Grippe|  Dr. Faye|           15|
|       4|         4|2024-01-08T10:00:00|consultation|     Suivi grossesse|Grossesse normale| Dr. Diouf|           30|
|       3|         3|2024-01-07T14:15:00|     ur

#### Normaliser la colonne date_visite

In [47]:
from pyspark.sql.functions import to_timestamp

df_visits = df_visits.withColumn(
    "date_visite",
    to_timestamp("date_visite", "yyyy-MM-dd'T'HH:mm:ss")
)


In [48]:
df_visits.show()

+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+
|visit_id|patient_id|        date_visite| type_visite|               motif|       diagnostic|   medecin|duree_minutes|
+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+
|       8|         8|2024-01-12 09:00:00|consultation|       Bilan sanguin|    Anémie légère|  Dr. Gaye|           20|
|       7|         7|2024-01-11 13:00:00|consultation|         Vaccination|              RAS|Dr. Camara|           10|
|       6|         6|2024-01-10 08:30:00|     urgence|Douleurs thoraciques|           Angine|Dr. Ndiaye|           35|
|       5|         5|2024-01-09 16:45:00|consultation|              Fièvre|           Grippe|  Dr. Faye|           15|
|       4|         4|2024-01-08 10:00:00|consultation|     Suivi grossesse|Grossesse normale| Dr. Diouf|           30|
|       3|         3|2024-01-07 14:15:00|     ur

#### ajout d'une colonne année, mois, jour

In [50]:
from pyspark.sql.functions import year, month, dayofmonth

df_visits = (
    df_visits
    .withColumn("annee", year("date_visite"))
    .withColumn("mois", month("date_visite"))
    .withColumn("jour", dayofmonth("date_visite"))
)


In [51]:
df_visits.show()

+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+-----+----+----+
|visit_id|patient_id|        date_visite| type_visite|               motif|       diagnostic|   medecin|duree_minutes|annee|mois|jour|
+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+-----+----+----+
|       8|         8|2024-01-12 09:00:00|consultation|       Bilan sanguin|    Anémie légère|  Dr. Gaye|           20| 2024|   1|  12|
|       7|         7|2024-01-11 13:00:00|consultation|         Vaccination|              RAS|Dr. Camara|           10| 2024|   1|  11|
|       6|         6|2024-01-10 08:30:00|     urgence|Douleurs thoraciques|           Angine|Dr. Ndiaye|           35| 2024|   1|  10|
|       5|         5|2024-01-09 16:45:00|consultation|              Fièvre|           Grippe|  Dr. Faye|           15| 2024|   1|   9|
|       4|         4|2024-01-08 10:00:00|consultation| 

In [52]:
path = "s3a://silver/visits"
df_visits.write.mode("overwrite").parquet(path)

# 🧱 Étapes de transformation Gold

#### Lecture de donneé apartir de la couche silver

In [53]:
bronze_path = "s3a://silver"
patient = spark.read.parquet(f"{bronze_path}/patients")
patient.show()

+-----------+----------+------+--------+--------------+--------------------+-------------+--------------------+----+-----------+-----------+---------+-------------+
|external_id|patient_id|   nom|  prenom|date_naissance|               email|     num_secu|             adresse|sexe|      ville|code_postal|telephone|date_creation|
+-----------+----------+------+--------+--------------+--------------------+-------------+--------------------+----+-----------+-----------+---------+-------------+
|     SRC001|         1|DIALLO| AMINATA|    1985-06-12|aminata.diallo@ex...|2850612345678|Rue des Manguiers 12|   F|      Dakar|      11000|771234567|   2020-01-15|
|     SRC002|         2|    BA|  MOUSSA|    1978-03-22|moussa.ba@example...|2780322456789|Avenue Blaise Diagne|   M|     ThiÃ¨s|      21000|772345678|   2019-11-03|
|     SRC003|         3|   SOW|   FATOU|    1990-09-10|fatou.sow@example...|2900912345678|      Rue 4, MÃ©dina|   F|      Dakar|      11000|773456789|   2021-05-20|
|     SRC0

In [54]:
bronze_path = "s3a://silver"
patient = spark.read.parquet(f"{bronze_path}/visits")
patient.show()

+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+-----+----+----+
|visit_id|patient_id|        date_visite| type_visite|               motif|       diagnostic|   medecin|duree_minutes|annee|mois|jour|
+--------+----------+-------------------+------------+--------------------+-----------------+----------+-------------+-----+----+----+
|       8|         8|2024-01-12 09:00:00|consultation|       Bilan sanguin|    Anémie légère|  Dr. Gaye|           20| 2024|   1|  12|
|       7|         7|2024-01-11 13:00:00|consultation|         Vaccination|              RAS|Dr. Camara|           10| 2024|   1|  11|
|       6|         6|2024-01-10 08:30:00|     urgence|Douleurs thoraciques|           Angine|Dr. Ndiaye|           35| 2024|   1|  10|
|       5|         5|2024-01-09 16:45:00|consultation|              Fièvre|           Grippe|  Dr. Faye|           15| 2024|   1|   9|
|       4|         4|2024-01-08 10:00:00|consultation| 

### Table Gold principale : table de faits

In [63]:
silver_path = "s3a://silver"
df_patients = spark.read.parquet(f"{silver_path}/patients")
df_visits = spark.read.parquet(f"{silver_path}/visits")

In [55]:
from pyspark.sql.functions import col

df_fact_visits = df_visits.select(
    col("visit_id"),
    col("patient_id"),
    col("date_visite"),
    col("type_visite"),
    col("motif"),
    col("diagnostic"),
    col("medecin"),
    col("duree_minutes"),
    col("annee"),
    col("mois"),
    col("jour")
)

In [60]:
gold_path = "s3a://gold"
df_fact_visits.write.mode("overwrite").parquet(f"{gold_path}/fact_visits")

Dimension Médecin (dim_medecin)

In [65]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [66]:
dim_medecin = df_visits.select(
    col("medecin")
).dropDuplicates()

# créer clé technique
dim_medecin = dim_medecin.withColumn(
    "medecin_id",
    monotonically_increasing_id()
)

# réorganiser colonnes
dim_medecin = dim_medecin.select(
    "medecin_id",
    "medecin"
)

In [67]:
dim_medecin.write.mode("overwrite").parquet(f"{gold_path}/dim_medecin")

##### Dimension Date (dim_date)

In [68]:
dim_date = df_visits.select(
    to_date("date_visite").alias("date")
).dropDuplicates()

dim_date = dim_date.withColumn("annee", year("date")) \
    .withColumn("mois", month("date")) \
    .withColumn("jour", dayofmonth("date")) \
    .withColumn("jour_semaine", dayofweek("date")) \
    .withColumn("trimestre", quarter("date"))

In [69]:
dim_date.write.mode("overwrite").parquet(f"{gold_path}/dim_date")

#### Dimension Patient (dim_patient)

In [71]:
dim_patient = df_patients.select(
    col("patient_id"),
    col("nom"),
    col("prenom"),
    col("sexe"),
    col("date_naissance"),
    col("ville")
).dropDuplicates(["patient_id"])


In [72]:
dim_patient.write.mode("overwrite").parquet(f"{gold_path}/dim_patient")